### OD-1. Исследование каналов привлечения

Задача:
1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.
2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

In [1]:
# Импорт библиотек
import json 
import requests
import pandas as pd 
from pprint import pprint

In [2]:
# Задаём токен
token = 'AQAEA7qhmZa4AAfQkYPU6RXAqERZnf9enrchyyI'
headers = {'Authorization': 'OAuth ' + token}

In [3]:
# Задаём словарь params с параметрами. Ключ словаря — это наименование параметра запроса, 
# а значение по ключу — значение этого параметра

params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions':'ym:s:lastsignTrafficSource,ym:s:hour,ym:s:isRobot',
          'date1': '2020-09-01',
          'date2': '2020-09-30',
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}

In [4]:
# Делаем запрос response. Тип нашего запроса GET, его можно выполнить с помощью функции requests.get(). 
# В неё мы передаём следующие параметры:
# - URL, по которому будем делать запрос
# - В аргументе params передаём заданные параметры запроса. Они хранятся в словаре params 
# - В аргументе headers передаем заголовки http, которые мы задали в словаре headers.

response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)

200


In [6]:
# Получить данные ответа можно с помощью функции json(), примененной к запросу response. 
# Это позволяет сразу же распознать формат json и переделать ответ из этого формата в объект Python.
# Запишем данные ответа в переменную metrika_data:

metrika_data = response.json()
#print(metrika_data.keys())

Прежде всего нас интересует значение по ключу data. Именно там содержатся данные статистики посещаемости, полученные от API Яндекс.Метрики.

In [7]:
# срез данных
pprint(metrika_data['data'][0:3])

[{'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '16', 'name': '16:00'},
                 {'id': 'no', 'name': 'People'}],
  'metrics': [189.0, 0.0]},
 {'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '15', 'name': '15:00'},
                 {'id': 'no', 'name': 'People'}],
  'metrics': [177.0, 0.0]},
 {'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '11', 'name': '11:00'},
                 {'id': 'no', 'name': 'People'}],
  'metrics': [169.0, 0.0]}]


Создаём датафрейм metrika_df из данных статистики metrika_data['data']:

In [8]:
metrika_df = pd.DataFrame(metrika_data['data'])
metrika_df.head()

,dimensions,metrics
0,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[189.0, 0.0]"
1,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[177.0, 0.0]"
2,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[169.0, 0.0]"
3,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[169.0, 0.0]"
4,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[167.0, 0.0]"


Создадим функцию, чтобы избавиться от вложенных словарей:

In [10]:
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:3])

[{'ym:s:hour': '16:00',
  'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 0.0,
  'ym:s:visits': 189.0},
 {'ym:s:hour': '15:00',
  'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 0.0,
  'ym:s:visits': 177.0},
 {'ym:s:hour': '11:00',
  'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 0.0,
  'ym:s:visits': 169.0}]


Создаём датафрейм из полученных данных:

In [11]:
metrika_df = pd.DataFrame(metrika_list_of_dicts)
metrika_df.head()

,ym:s:lastsignTrafficSource,ym:s:hour,ym:s:isRobot,ym:s:visits,ym:s:robotPercentage
0,Search engine traffic,16:00,People,189.0,0.0
1,Search engine traffic,15:00,People,177.0,0.0
2,Search engine traffic,11:00,People,169.0,0.0
3,Search engine traffic,12:00,People,169.0,0.0
4,Search engine traffic,14:00,People,167.0,0.0


In [13]:
# переименуем колонки

metrika_df.columns=['Source','Hour','IsRobot','Visits','RobotPercentage']
metrika_df.head()

,Source,Hour,IsRobot,Visits,RobotPercentage
0,Search engine traffic,16:00,People,189.0,0.0
1,Search engine traffic,15:00,People,177.0,0.0
2,Search engine traffic,11:00,People,169.0,0.0
3,Search engine traffic,12:00,People,169.0,0.0
4,Search engine traffic,14:00,People,167.0,0.0


Оценим количество ботов, которые заходят на наш сайт, и их долю от всего трафика.

In [18]:
# сводная таблица по визитам людей и роботов (столбец IsRobot)
pt_robot_visits = metrika_df.pivot_table(values = ['Visits'], columns = ['IsRobot'], aggfunc = 'sum')
# процент ботов от общего трафика
pt_robot_visits['robot_visits_%'] = round(pt_robot_visits['Robots'] * 100 / (pt_robot_visits['Robots'] + pt_robot_visits['People']), 2)
pt_robot_visits.head()

IsRobot,People,Robots,robot_visits_%
Visits,3353.0,962.0,22.29


ВЫВОД: 22.29 % от всего трафика составляет количество ботов.

Определим, с каких источников чаще всего заходят роботы:

In [20]:
# сводная таблица по визитам для источников трафика
mask = metrika_df['IsRobot'] == 'Robots'
pt_source_visits = metrika_df[mask].pivot_table(values = ['Visits'], columns = ['Source'], aggfunc = 'sum')
pt_source_visits.head()

Source,Cached page traffic,Direct traffic,Internal traffic,Link traffic,Search engine traffic,Social network traffic
Visits,1.0,177.0,3.0,31.0,739.0,11.0


Промежуточный вывод: роботы чаще всего заходят с источника "Search engine traffic".

Определим, в какое время суток чаще всего заходят роботы.

In [33]:
mask_source = metrika_df['Source'] == 'Search engine traffic'
pt_hour_visits = metrika_df[mask_source & mask].pivot_table(values = ['Visits'], index = ['Hour'], aggfunc = 'sum')
print(pt_hour_visits['Visits'].sort_values(ascending=False))

Hour
13:00    64.0
15:00    63.0
16:00    62.0
11:00    59.0
14:00    58.0
12:00    56.0
17:00    55.0
19:00    38.0
09:00    36.0
10:00    36.0
18:00    31.0
22:00    30.0
21:00    29.0
08:00    29.0
20:00    26.0
23:00    21.0
00:00    17.0
01:00     8.0
07:00     6.0
02:00     5.0
06:00     3.0
04:00     3.0
03:00     3.0
05:00     1.0
Name: Visits, dtype: float64


ВЫВОД: роботы чаще всего заходят с источника "Search engine traffic" в 13:00. Промежуток времени, в котором они чаще всего заходят отватывает с 11:00 по 17:00.